# Pandas ilə SQLvari sorğuların yazılması
![head.jpg](https://github.com/limpapud/data_science_tutorials_projects/raw/master/DataScience_Tutorials/AZ/assets/head.jpg)

SQL ilə heç olmasa qismən tanışlığı olan adam *"SQL-in əsasların bir neçə saat ərzində öyrənib ilk sorğuları yazmaq olar"* cümləsi ilə razılaşar (hər halda mən bu cür fikirləşirəm). Python ilə də eynən, bu dil ən sadə və proqramlaşdırmanı öyrənməyə başlayanlar üçün ideal dil sayılır.
İndi, verilənlər elmi və məlumat analizi ən trend mövzu olan vaxtda Python-un **Pandas** modulu vasitəsi ilə SQL kimi məlumat emal etmə məqaləsi elə lap yerinə düşərdi, və budur o!

Jurnalda SQL-da DML (Məlumat Manipulyasiyası Dili) vasitəsi ilə icra olunan məlumatın sorğulanması (SELECT), məlumatın daxil olunması (INSERT), məlumat pozulması (DELETE) və məlumat yeniləməsi (UPDATE) komandalarının Pandas analoqları təqdim olunacaq. Hər iki variantda sorğular icra olunacaq və göstəriləcək.

#### Başlayaq!
İşə Pandasın importundan başlayırıq:

In [1]:
import pandas as pd

Jurnalın daxilində 2 cədvəldən istifadə olunacaq və onlar mənim GitHub repositoriyamda yerləşən **dummy_data.xlsx** Excel faylının ayrı-ayrı səhifələrinə yerləşir.
* **staff** - İşçinin *"unikal identifikatoru", "adı", "soyadı", "cinsi", "küçə", "şirkət", "vəzifəsi"*
* **staff_add_info** - *"unikal identifikatoru", "email ünvanı", "aktivlik statusu", "istifadəçi adı", "MD5-də saxlanan şifrəsi", "mobil_nömrəsi"*

Cədvəllərin ortaq identifikatoru - "guid" sütunudur, tutuşdurma baş verdikdə bu sütundan istifadə edəcəyik

İşləyəcəyimiz cədvəllər haqqında ümumi məlumat aldıq, datafreymlərə keçirmək olar:
Hər sütuna uyğun eyni adlı datafreymlər yaratmaqdan başlayırıq. Bunun üçün *pd.read_excel* funksiyası vasitəsi ilə excel faylın müxtəlif səhifələrin "oxuyub" yaradırıq.

In [2]:
# ilk öncə açacağımız fayla link, sonra isə səhifənin adın qeyd edirik
staff_df = pd.read_excel('https://raw.githubusercontent.com/limpapud/datasets/master/Tutorial_datasets/SQL_Pandas.xlsx',sheet_name='staff') 
staff_add_info = pd.read_excel("https://raw.githubusercontent.com/limpapud/datasets/master/Tutorial_datasets/SQL_Pandas.xlsx",sheet_name='staff_add_info')

In [3]:
staff_df.head(3) # head(3) Məlumatın ilk 3 sətrin əks etdirir

,guid,first_name,last_name,gender,street,company,job_title
0,6a9472d4-4949-4f98-8543-c6878d558422,Rhoda,Halbert,Female,NaN,Zooveo,Registered Nurse
1,3e89c11d-fd66-427f-a294-029fde1a4e40,Nial,Doggett,Male,Londonderry,Demizz,Account Executive
2,8c37f0ca-75c5-42f9-8f45-92f1fc2cb027,Valentina,Stenners,Female,Tony,Wikizz,Safety Technician I


In [4]:
staff_add_info.head(3)

,guid,email,a_status,username,password,phone_number
0,6a9472d4-4949-4f98-8543-c6878d558422,rhalbert0@mashable.com,False,rhalbert0,99a0837db38c3fb198371fc78336f1ec,+62 (345) 865-9966
1,3e89c11d-fd66-427f-a294-029fde1a4e40,ndoggett1@odnoklassniki.ru,True,ndoggett1,e6cb5bfd91b8fe6f750837b5c669cb2a,+7 (947) 890-1005
2,8c37f0ca-75c5-42f9-8f45-92f1fc2cb027,vstenners2@umn.edu,True,vstenners2,2c6ac4200d1e8efae067e35eb61ba349,+62 (373) 436-5895


Pandas modulu ilə işin nümaişi 2 hissədən ibarət olacaq:
1. Real SQL əmr sorğusunun yazılması və icrası
2. Yazılmış sorğunun analoqunun datafreymlər üzərində Pandas vasitəsi ilə hazırlanması

### SELECT

SQL-də SELECT əmri məlumatın sorğulanması və əldə edilməsi üçün istifadə olunur.

Deyək ki biz *staff_db* cədvəlindən bütün sütun və sətrləri əldə etmək istəyirik. Bunun üçün yazılmış SQL sorğusu aşağıdaki kimidir:

**SELECT * FROM staff_db**

Pandas vasitəsi ilə bu komandanı sadəcə datafreymin adın yazmaq ilə icra edə bilərsiniz.

In [5]:
staff_df

,guid,first_name,last_name,gender,street,company,job_title
0,6a9472d4-4949-4f98-8543-c6878d558422,Rhoda,Halbert,Female,NaN,Zooveo,Registered Nurse
1,3e89c11d-fd66-427f-a294-029fde1a4e40,Nial,Doggett,Male,Londonderry,Demizz,Account Executive
2,8c37f0ca-75c5-42f9-8f45-92f1fc2cb027,Valentina,Stenners,Female,Tony,Wikizz,Safety Technician I
3,0215a527-9df4-43fa-97ca-bc580ac56a70,Gamaliel,Coggell,Male,Oak,Devbug,Senior Financial Analyst
4,056e35a8-46bf-4886-98ac-3a06b625d036,Pru,Wretham,NaN,NaN,Blogtag,Research Assistant III
5,2764b956-a17b-4505-a70c-55e8112b2420,Franky,Philipps,Female,Hallows,Skyba,Actuary
6,581aae0b-e53c-49a1-9d2b-d4b316a2a7bb,Cammy,Veltmann,NaN,Moulton,Demizz,Information Systems Manager
7,37a2bbe8-0e00-41e0-8661-579771028957,Tedd,Delahunty,Male,2nd,Kare,Associate Professor
8,8de537ba-c94e-40c2-bd00-fc51a80abdc7,Remy,Greschke,Female,Derek,BlogXS,Senior Editor
9,806ebef0-6d82-4efb-9d8b-de8d86dbd29e,Seward,Lambeth,Male,Kensington,Livetube,Programmer III


**Ups!** Nədənsə (*əslində əlbəttə ki Pandasın standart ayarlarına uyğun, perfomans və praktiklik baxımından*) məlumat öyrəşdiyimiz SQL sorğulama zamanı kimi tam olaraq yox, hissə-hissə olaraq çıxdı, yəni yalnız ilk 30 və son 30 sətr əks olundu.

Bu məqamı nəzərə alaraq ayarlarda maksimal sətr və sütun sayın dəyişək və limitsiz edək. (***Diqqət! Real həyatda böyük həcmli verilənlər ilə iş zamanı bunu etməyin!!!***)

Maksimal sütun və sətr sayın *None* qeyd edib limitləri aradan qaldırırıq.

In [8]:
pd.set_option('display.max_columns',7) # sütün sayı üzərindən limit qaldıran əmr
pd.set_option('display.max_rows',60) # sətr sayı üzərindən limit qaldıran əmr

Yenidən yoxlayaq ...

In [7]:
staff_df

,guid,first_name,last_name,gender,street,company,job_title
0,6a9472d4-4949-4f98-8543-c6878d558422,Rhoda,Halbert,Female,NaN,Zooveo,Registered Nurse
1,3e89c11d-fd66-427f-a294-029fde1a4e40,Nial,Doggett,Male,Londonderry,Demizz,Account Executive
2,8c37f0ca-75c5-42f9-8f45-92f1fc2cb027,Valentina,Stenners,Female,Tony,Wikizz,Safety Technician I
3,0215a527-9df4-43fa-97ca-bc580ac56a70,Gamaliel,Coggell,Male,Oak,Devbug,Senior Financial Analyst
4,056e35a8-46bf-4886-98ac-3a06b625d036,Pru,Wretham,NaN,NaN,Blogtag,Research Assistant III
5,2764b956-a17b-4505-a70c-55e8112b2420,Franky,Philipps,Female,Hallows,Skyba,Actuary
6,581aae0b-e53c-49a1-9d2b-d4b316a2a7bb,Cammy,Veltmann,NaN,Moulton,Demizz,Information Systems Manager
7,37a2bbe8-0e00-41e0-8661-579771028957,Tedd,Delahunty,Male,2nd,Kare,Associate Professor
8,8de537ba-c94e-40c2-bd00-fc51a80abdc7,Remy,Greschke,Female,Derek,BlogXS,Senior Editor
9,806ebef0-6d82-4efb-9d8b-de8d86dbd29e,Seward,Lambeth,Male,Kensington,Livetube,Programmer III


**Əla.** Davam etmək olar.

Növbəti məqsədimiz yalnız tələb olunan sütunları sorğulayıb 3 sətri əks etdirməkdir

SQL versiya: **SELECT TOP(3) first_name, last_name, gender, job_title FROM staff_db**

Pandas versiyasında sorğunun məntiqi datafreym adından sonra tələb olunan sütun adların siyahı vasitəsi ilə təmin edərək *.head()* funksiyası köməkliyi ilə **ilk** 3 yazını əks etdirməkdir.

In [9]:
staff_df[['first_name','last_name','gender','job_title']].head(3)

,first_name,last_name,gender,job_title
0,Rhoda,Halbert,Female,Registered Nurse
1,Nial,Doggett,Male,Account Executive
2,Valentina,Stenners,Female,Safety Technician I


İlkin sorğu etməyə öyrəndik, növbə gəldi filterlərin əlavə olunmasına.

SQL versiya: **SELECT TOP(3) first_name, last_name, gender, job_title FROM staff_db WHERE gender = 'Male'**

Pandas-da datafreym daxili filter əlavə etmək üçün **binar "maska"-dan** istifadə etməliyik. Amma nədir binar maska? O filterlənən sütun üzrə keçərək sütundakı hər bir sətr üzrə dəyərin tələb olunan dəyərə bərabər olub olmadığını yoxlayıb müvafiq olaraq True/False (Doğru\Yalnız) dəyəri verir. Sonra isə bu maska üzrə sütun üzrə əks olunan dəyərləri filterləmək mümkün olur.

Məsəl üçün Kişi cinsi üzrə maska yaradaq və əks etdirək:

In [10]:
binar_maska = staff_df['gender']=='Male'
binar_maska.head(5)

0    False
1     True
2    False
3     True
4    False
Name: gender, dtype: bool

Gördüyümüz kimi **binar_maska** 1 ölçülü datafreym yarandı və realda "Male" dəyəri olan sətrdə "True", digər dəyərlər olan sətrlərdə isə "False" dəyərin saxlayır. İndi isə bu maskanı tətbiq edək və cavabları yoxlayaq:

In [11]:
staff_df[['first_name','last_name','gender','job_title']][staff_df['gender']=='Male'].head(3)

,first_name,last_name,gender,job_title
1,Nial,Doggett,Male,Account Executive
3,Gamaliel,Coggell,Male,Senior Financial Analyst
7,Tedd,Delahunty,Male,Associate Professor


Növbəti addım - istifadə olunan filterləri artıraq və zənqinləşdirməkdir. Elə sorğu yazaq ki *yalnız cinsi 'Kişi'*  **VƏ** *vəzifəsi 'Account Executive' olan* **VƏ YA** *adı 'Valentina' olan işçilər* siyahısı çıxsın.

SQL versiya: **SELECT TOP(3) first_name, last_name, gender, job_title FROM staff_db WHERE (gender = 'Male' AND job_title = 'Account Executive') OR (first_name = 'Valentina')**

Pandas-da **VƏ** məntiqi ampersand ("***&***") vasitəsi ilə, **VƏ YA** məntiqi isə vertikal xətt ("**|**") vasitəsi ilə tənzimlənir.

In [12]:
staff_df[['first_name','last_name','gender','job_title']][((staff_df['gender']=='Male') & (staff_df['job_title']=='Account Executive')) | (staff_df['first_name']=='Valentina')].head(3)

,first_name,last_name,gender,job_title
1,Nial,Doggett,Male,Account Executive
2,Valentina,Stenners,Female,Safety Technician I
115,Brok,Anderer,Male,Account Executive


### UPDATE

SQL-də UPDATE əmri məlumatın dəyişdirilməsi və yenilənməsi üçün istifadə olunur.

SQL versiyası: **UPDATE staff_df SET gender = 'Male' WHERE (gender = 'Male' AND job_title = 'Account Executive') OR (first_name = 'Valentina'**)

Pandas ilə sətrləri yeniləmək üçün index ilə işləmək üçün *.loc* və yenə binar maskalardan istifadə edəcəyik.

Məntiq aşağıdaki kimidir: DataFreym_adı.loc[binar maska, dəyişməli sütun] = 'hədəf dəyər'

In [13]:
staff_df.loc[((staff_df['gender']=='Male') & (staff_df['job_title']=='Account Executive')) | (staff_df['first_name']=='Valentina'),'gender']='Male'

In [14]:
staff_df.head(3)

,guid,first_name,last_name,gender,street,company,job_title
0,6a9472d4-4949-4f98-8543-c6878d558422,Rhoda,Halbert,Female,NaN,Zooveo,Registered Nurse
1,3e89c11d-fd66-427f-a294-029fde1a4e40,Nial,Doggett,Male,Londonderry,Demizz,Account Executive
2,8c37f0ca-75c5-42f9-8f45-92f1fc2cb027,Valentina,Stenners,Male,Tony,Wikizz,Safety Technician I


Dəyişiklikləri yoxlayaq. Bunun üçün filtr üzrə yazılmış yazılarda 'gender' sütunun yoxlayaq.

In [15]:
staff_df[['first_name','last_name','gender','job_title']][((staff_df['gender']=='Male') & (staff_df['job_title']=='Account Executive')) | (staff_df['first_name']=='Valentina')].head(3)

,first_name,last_name,gender,job_title
1,Nial,Doggett,Male,Account Executive
2,Valentina,Stenners,Male,Safety Technician I
115,Brok,Anderer,Male,Account Executive


 2-ci indeksdə 'Valentina' adlı istifadəçi sorğulanan parameterlərə uyğun gəldi və "Female"-dən "Male"-yə dəyişdi.

### INSERT

SQL-də INSERT əmri məlumatın əlavə olunması üçün istifadə olunur.

Nümaiş üçün staff_df-də yeni işçi əlavə edək:

SQL variantı: ***INSERT INTO staff_df ('guid', 'first_name', 'last_name', 'gender', 'street', 'company', 'job_title') VALUES ('6a9472d4-4949-4f98-8543-c6878d558422', 'Omar', 'Bayramov', 'Male', NULL, 'Limpapud', NULL)***

Pandas vasitəsi ilə isə eyni nəticə almaq üçün artıq tanış olan *.loc* indeks köməkçisindən istifadə edərək sətr əlavə edirik.

Pandas variantı: ***DatafreymAdı.loc[ indeks ] = ['vergül', 'ilə', 'ayrılmış', 'dəyərlər']***

İndeksi əldə etmək üçün datafreymin uzunluğun ölçüb uzunluğunu dəyər kimi ".loc"-a ötürək.

In [16]:
len(staff_df)

1000

In [17]:
staff_df.loc[len(staff_df)] = ['6a9472d4-4949-4f98-8543-c6878d558422','Omar','Bayramov','Male','NaN','Limpapud','NaN']

In [18]:
staff_df.tail(3) # son yazıları yoxlayaq

,guid,first_name,last_name,gender,street,company,job_title
998,30a8515e-92ff-4e6f-9d59-8e941bb1cf46,Lemuel,Cardoe,Male,Corben,Topicware,VP Sales
999,c616bbaf-32b8-4942-ab23-e16e002cf08b,Jo,Brewis,Female,Old Shore,Npath,Physical Therapy Assistant
1000,6a9472d4-4949-4f98-8543-c6878d558422,Omar,Bayramov,Male,NaN,Limpapud,NaN


## DELETE

SQL-də DELETE əmri məlumatın silinməsi üçün istifadə olunur.

Hər-hansı bir məlumatı datafreymdən silmək istədikdə SQL-a əks məntiqdən istifadə etməliyik. Nümaiş üçün az əvvəl əlavə etdiyimiz "Ömər" və ayrı "Ömər" işçilərin silək.

SQL variantı: ***DELETE FROM staff_db WHERE first_name = 'Omar'***

Yəni əgər SQL sorğu "***staff_db-dən 'Ömər' adlı işçiləri poz***" əmrin verirsə Pandas sorğusu "***staff_db-də "Ömər" adlı işçilərdən başqa hamısın saxla*** məntiqi istifadə edir.

In [19]:
staff_df = staff_df.loc[staff_df['first_name'] != 'Omar']
staff_df.tail(3)

,guid,first_name,last_name,gender,street,company,job_title
997,31d88f81-976c-4dc2-a656-f21de7129ef7,Chandler,Watchorn,Male,Huxley,Skimia,Cost Accountant
998,30a8515e-92ff-4e6f-9d59-8e941bb1cf46,Lemuel,Cardoe,Male,Corben,Topicware,VP Sales
999,c616bbaf-32b8-4942-ab23-e16e002cf08b,Jo,Brewis,Female,Old Shore,Npath,Physical Therapy Assistant


Hazırdır! Məlumat pozuldu.

### Jurnalın sonu

Jurnalın sonuna çatdınız. Oxuduğunuz üçün təşəkkürlər. Bu məqalə periodik olaraq yenilənəcək və əlavələr qəbul edəcək. Sizin əlavəniz, təklifiniz, iradınız olduğu halda GitHub vasitəsi ilə "İssue" yaradaraq və ya aşağıda qeyd olunmuş əlaqə vasitələri ilə fikrinizi bildirə bilərsiniz.

### Əlaqə

Müəllif ilə əlaqə omarbayramov@hotmail.com elektron ünvan üzərindən aparıla bilər. Əlavə olaraq sosial şəbəkə və digər saytlara linklər əlavə olunur.
[Facebook](https://www.facebook.com/Omar.X.Bayramov)
[Wordpress Blog](https://omarbayramov.wordpress.com/)
[LinkedIn](https://www.linkedin.com/in/omarbayramov/)
